<a href="https://colab.research.google.com/github/Czyundream/Leetcodetest/blob/main/%E8%AF%AF%E5%B7%AE%E8%A1%A5%E5%81%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#导入常用库
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l


In [18]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
import pandas as pd

In [5]:
#读取文件
!pip install -u -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':'1C2cnS5fi2Du6UcCbShpj1W2AaFD49huD'})
downloaded.GetContentFile('250HZ.xlsx')



Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [6]:
def xlsx_to_csv():
    data_xls=pd.read_excel('250HZ.xlsx')
    data_xls.to_csv('250HZ.csv',encoding='utf-8')

xlsx_to_csv()

In [7]:
data = pd.read_csv('250HZ.csv')
print(data)

      Unnamed: 0       光电           磁         温度        误差
0              0  358.375  359.331207  17.201250  0.956207
1              1  357.750  358.604736  17.242500  0.854736
2              2  357.000  357.823334  17.366249  0.823334
3              3  356.250  357.070068  17.325001  0.820068
4              4  355.500  356.318909  17.407499  0.818909
...          ...      ...         ...        ...       ...
1904        1904    1.125    1.896515  17.118750  0.771515
1905        1905    0.750    1.599884  17.283751  0.849884
1906        1906    0.500    1.349945  17.242500  0.849945
1907        1907    0.375    1.239395  17.283751  0.864395
1908        1908    0.000    0.819855  17.160000  0.819855

[1909 rows x 5 columns]


In [13]:
inputs , outputs= data.iloc[477: 1909, 2 : 4] , data.iloc[477 : 1909, 1]
features, labels = torch.tensor(inputs.values), torch.tensor(outputs.values)

In [26]:
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [36]:
def load_array(data_arrays, batch_size, is_train=True):
    '''构造一个PyTorch数据迭代器'''
    dataset = data.TensorDataset(*data_arrays)#将数据转为dataset，*表示将元组拆解为一维数字作为传入参数
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
    #将dataset型数据挑选batch_size大小的样本，shuffle控制是否打乱

batch_size = 50
data_iter = load_array((features, labels), batch_size)

next(iter(data_iter))

[tensor([[ 15.4413,  17.4075],
         [ 47.4348,  17.1600],
         [299.0692,  17.3662],
         [ 42.7780,  17.2425],
         [176.6911,  17.2838],
         [139.8896,  17.4075],
         [219.3592,  17.3250],
         [ 61.1643,  17.2838],
         [234.1592,  17.3250],
         [138.0961,  17.4075],
         [202.4080,  17.2425],
         [132.2562,  17.1187],
         [146.5130,  17.2838],
         [290.0576,  17.2838],
         [112.2130,  17.2838],
         [272.8688,  17.2838],
         [112.4300,  17.1600],
         [ 17.0700,  17.1600],
         [249.5860,  17.2838],
         [314.9492,  17.2013],
         [349.7882,  17.4075],
         [142.9589,  17.3662],
         [ 85.9879,  17.3250],
         [144.2917,  17.2838],
         [124.4030,  17.2013],
         [  6.8623,  17.2013],
         [228.7889,  17.4487],
         [276.4037,  17.3662],
         [196.6285,  17.2425],
         [181.0870,  17.2013],
         [309.2679,  17.2013],
         [317.4053,  17.2838],
        

In [20]:
from torch import nn#nn为神经网络的缩写

net = nn.Sequential(nn.Linear(2, 1))#Linear为全连接层，需要指定输入维度和输出维度
#Sequential可以理解为容器，即list of layers

In [33]:
net[0].weight.data.normal_(1, 0.01)#normal_为使用正态分布替换掉data的值
net[0].bias.data.fill_(0)#bias为偏差

tensor([0.])

In [34]:
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr=0.003)#optim.SGD需要指定两个参数：net.parameters()为拿出所有参数，lr学习率

In [40]:
num_epochs = 20
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch{epoch + 1}, loss{l:f}')#l:f打印l，格式为浮点数

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([50])) that is different to the input size (torch.Size([50, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1432])) that is different to the input size (torch.Size([1432, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input

epoch1, lossnan
epoch2, lossnan
epoch3, lossnan
epoch4, lossnan
epoch5, lossnan
epoch6, lossnan
epoch7, lossnan
epoch8, lossnan
epoch9, lossnan
epoch10, lossnan
epoch11, lossnan
epoch12, lossnan
epoch13, lossnan
epoch14, lossnan
epoch15, lossnan
epoch16, lossnan
epoch17, lossnan
epoch18, lossnan
epoch19, lossnan
epoch20, lossnan
